In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from scipy.stats import linregress
import json

# Import API key
from api_key import weather_api_key
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Read CSV lat/long
lat_long_df = pd.read_csv('fatal_alligator_attacks_US_with_latlng.csv')
lat_long_df.head()

,Date,UNIX_start_date,UNIX_end_date,First Name,Last Name,Age,Sex,City,State,Lat,Lng,Details
0,2022-08-15,1.660522e+09,1.660522e+09,Nancy,Becker,88,female,Hilton Head Island,South Carolina,32.22,-80.75,The victim fell into a pond in a gated communi...
1,2022-07-15,1.657843e+09,1.657843e+09,Rose Marie,Wiegand,81,female,Englewood,Florida,26.96,-82.35,The victim fell into a pond along a golf cours...
2,2022-06-24,1.656029e+09,1.656029e+09,Micheal,Burstein,75,male,Myrtle Beach,South Carolina,33.69,-78.89,The victim was dragged into a pond in Myrtle B...
3,2022-05-31,1.653955e+09,1.653955e+09,Sean,McGuinness,47,male,Largo,Florida,27.91,-82.79,The victim was attacked while trying to retrie...
4,2021-08-30,1.630282e+09,1.630282e+09,Timothy,Satterlee,71,male,Slidell,Louisiana,30.28,-89.78,The victim was attacked in floodwaters of Hurr...


In [41]:
lat_long_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             41 non-null     object 
 1   UNIX_start_date  38 non-null     float64
 2   UNIX_end_date    38 non-null     float64
 3   First Name       41 non-null     object 
 4   Last Name        40 non-null     object 
 5   Age              41 non-null     object 
 6   Sex              41 non-null     object 
 7   City             41 non-null     object 
 8   State            41 non-null     object 
 9   Lat              41 non-null     float64
 10  Lng              41 non-null     float64
 11  Details          41 non-null     object 
dtypes: float64(4), object(8)
memory usage: 4.0+ KB


In [42]:
url='https://archive-api.open-meteo.com/v1/era5?latitude='

weather = []

print('Beginning Data Retrieval')     
print('-----------------------------')
for x in range(len(lat_long_df)):
    lat = lat_long_df.iloc[x,9]
    lon = lat_long_df.iloc[x,10]
    date = lat_long_df.iloc[x,0]
    
    response=requests.get(f"{url}{lat}&longitude={lon}&start_date={date}&end_date={date}&daily=rain_sum&daily=temperature_2m_max&timezone=CST&temperature_unit=fahrenheit").json()
        
    try:
        lat_api = response['latitude']
        lon_api = response['longitude']
        rain = response['daily']['rain_sum']
        temp = response['daily']['temperature_2m_max']
        
    except KeyError:
             print('not found. Skipping...')

    weather.append({
        'Lat': lat_api,
        'Lng': lon_api,
        'Max Temp (F)': temp,
        'Rain (mm)' : rain})


    

Beginning Data Retrieval
-----------------------------


JSONDecodeError: [Errno Expecting value] <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>
: 0

In [ ]:
weather_df = pd.DataFrame(weather)
weather_df

,Lat,Lng,Max Temp (F),Rain (mm)
0,32.25,-80.75,[86.4],[0.0]
1,27.00,-82.25,[90.3],[0.8]
2,33.75,-79.00,[83.8],[0.0]
3,28.00,-82.75,[83.0],[1.0]
4,30.25,-89.75,[82.6],[72.6]
5,32.75,-80.25,[75.7],[0.0]
6,32.25,-80.75,[88.7],[0.2]
7,26.00,-80.25,[83.9],[6.0]
8,32.75,-80.00,[97.3],[0.0]
9,28.50,-81.50,[93.8],[1.7]


In [ ]:
weather_df["Max Temp (F)"] = weather_df["Max Temp (F)"].apply(lambda x:x.replace('[',''))


In [ ]:
weather_df

,Lat,Lng,Max Temp (F),Rain (mm)
0,32.25,-80.75,[],[0.0]
1,27.00,-82.25,[],[0.8]
2,33.75,-79.00,[],[0.0]
3,28.00,-82.75,[],[1.0]
4,30.25,-89.75,[],[72.6]
5,32.75,-80.25,[],[0.0]
6,32.25,-80.75,[],[0.2]
7,26.00,-80.25,[],[6.0]
8,32.75,-80.00,[],[0.0]
9,28.50,-81.50,[],[1.7]
